## 0. Setting up workenvironment<a id="0"></a>

In [ ]:
suppressPackageStartupMessages({
    library(DropletUtils)
    library(SingleCellExperiment)
    library(scuttle)
    library(Seurat)
    library(SeuratWrappers)
    library(stringr)
    library(dplyr)
    library(data.table)
    library(Matrix)
    library(patchwork)
    library(ggplot2)
})

options(repr.plot.width = 16, repr.plot.height = 8)

## 1. Importing data<a id="1"></a>


In [ ]:
seurat.data <- Read10X(data.dir = "/home/jovyan/researcher_home/Documents/Tom/Atlas/data/raw_data/Atlas_v3/Nookaew_2024/Sp7periosteum1/cellranger/outs/filtered_feature_bc_matrix/")
seurat <- CreateSeuratObject(counts = seurat.data)
rm(seurat.data)

## 2. Preprocessing: quality control with scater<a id="4"></a>

### 2.1. RD3_1

#### 2.1.1. Converting a Seurat object to SingleCellExperiment 

In [ ]:
sce <- as.SingleCellExperiment(seurat)

#### 2.1.2. Calculating QC metrics

In [ ]:
is.mito <- grepl("^mt-", rownames(sce))
mito.list <-  grep("^mt-", rownames(sce), value = TRUE)

In [ ]:
mito.list

In [ ]:
is.spike <- grepl("^gSpikein", rownames(sce))
spike.list <- grep("^gSpikein", rownames(sce), value = TRUE)

In [ ]:
celldata <- perCellQCMetrics(sce, subsets = list(Mt = is.mito, ERCC = is.spike))

In [ ]:
sce <- addPerCellQC(sce, subsets=list(Mito = is.mito, ERCC = is.spike))
colnames(colData(sce))

Expression of all ERCC spikeins is 0 because they were not used in this study.

##### **2.1.2.1. QC of the cells by feature number and library size**


In [ ]:
libsize.drop <- isOutlier(sce$sum, nmads = 3, type = "both", log = TRUE)
feature.drop <- isOutlier(sce$detected, nmads = 3, type = "both", log = TRUE)

# libsize.drop cutoff values
min.libsize <- (median(sce$sum) - 3*mad(sce$sum))/1e3
max.libsize <- (median(sce$sum) + 3*mad(sce$sum))/1e3

# feature.drop cutoff values
min.features <- (median(sce$detected) - 3*mad(sce$detected))
max.features <- (median(sce$detected) + 3*mad(sce$detected))

par(mfrow = c(1,2))

hist(sce$sum/1e3, xlab = "Library sizes (thousands)", main = "", breaks = 20,
     col = "grey", ylab = "Number of cells")

abline(v = min.libsize, col = "blue", lwd = 2, lty = 2)
abline(v = max.libsize, col = "blue", lwd = 2, lty = 2)

hist(sce$detected, xlab = "Number of expressed genes", main = "", breaks = 20,
     col = "grey", ylab = "Number of cells")

abline(v = min.features, col = "blue", lwd = 2, lty = 2)
abline(v = max.features, col = "blue", lwd = 2, lty = 2)

##### **2.1.2.2. Proportion of mitochondrial reads**


In [ ]:
mito.drop <- isOutlier(sce$subsets_Mito_percent, nmads = 3)

max.mito <- median(sce$subsets_Mito_percent) + 3*mad(sce$subsets_Mito_percent)

hist(sce$subsets_Mito_percent, xlab = "Mitochondrial proportion (%)",
     ylab = "Number of cells",
     breaks = 40, main = "", col = "grey")

abline(v = max.mito, col = "blue", lwd = 2, lty = 2)

Subset by column to retain only high-quality cells that pass all filters.

In [ ]:
sce_QC <- sce[,!(libsize.drop | feature.drop | mito.drop)]
data.frame(ByLibrarySize = sum(libsize.drop), ByFeature = sum(feature.drop), ByMito = sum(mito.drop),
           Remaining = ncol(sce_QC))

#### 2.1.2. Filtering out low-abundance genes


In [ ]:
average.counts <- calculateAverage(sce_QC, exprs_values = "counts",
                                   subset_row = NULL)
genes.to.keep <- average.counts >= 0.001
sum(genes.to.keep)

In [ ]:
hist(log10(average.counts), breaks = 100, main = "", col = "grey",
     xlab = expression(Log[10]~"average count"), ylab = "Gene counts")
abline(v = log10(0.001), col = "blue", lwd = 2, lty = 2)

rm(average.counts)

Apply the mean-based filter to the data by subsetting the `SingleCellExperiment` object.

In [ ]:
sce <- sce_QC[genes.to.keep,]

rm(sce_QC)

dim(sce)

#### 2.1.3. Duplicate removal

In [ ]:
sce <- sce[!duplicated(rownames(sce)),]
dim(sce)

In [ ]:
sce <- as.Seurat(sce)

In [ ]:
sce

In [ ]:
saveRDS(sce, file = "/home/jovyan/researcher_home/Documents/Tom/Atlas/data/SCA_core/Obrien/QC_sp7_periost_1.rds")